In [7]:
!pip install spektral
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
import networkx as nx
from spektral.layers import GCNConv # spektral is now installed and can be imported
from spektral.data import Dataset, Graph
from spektral.data.loaders import SingleLoader
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# ... (rest of your code)
# Stage 2: Loading and Preparing the Dataset
data = pd.read_csv('/content/so2_20180101_20231031 (1).csv')

#select the column 'V' for prediction
selected_column='33204020'


# Split the data into training and testing sets
train_data,test_data=train_test_split(data[selected_column],test_size=0.2,shuffle=False)
# Convert to numpy arrays
train_data = train_data.values
test_data = test_data.values

# Stage 3: Data Normalization
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data.reshape(-1, 1)).flatten()
test_data = scaler.transform(test_data.reshape(-1, 1)).flatten()

# Stage 4: Data Preparation
def create_dataset(data, look_back=10):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        a = data[i:(i + look_back)]
        X.append(a)
        Y.append(data[i + look_back])
    return np.array(X), np.array(Y)

look_back = 10
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

# Stage 5: Data Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], look_back, 1))
X_test = np.reshape(X_test, (X_test.shape[0], look_back, 1))

# Stage 6: Building and Training the Model
class SO2Dataset(Dataset):
    def __init__(self, X, y, **kwargs):
        self.X = X
        self.y = y
        super().__init__(**kwargs)

    def read(self):
        graphs = []
        for i in range(len(self.X)):
            adj = nx.adjacency_matrix(nx.path_graph(self.X[i].shape[0]))
            x = self.X[i]
            y = self.y[i]
            graphs.append(Graph(x=x, a=adj, y=y))
        return graphs

train_dataset = SO2Dataset(X_train, y_train)
test_dataset = SO2Dataset(X_test, y_test)

class GNNModel(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(32, activation='relu')
        self.dropout = Dropout(0.5)
        self.dense1 = Dense(16, activation='relu')
        self.dense2 = Dense(1)

    def call(self, inputs):
        x, a = inputs
        x = self.conv1([x, a])
        x = self.dropout(x)
        x = self.dense1(x)
        return self.dense2(x)

model = GNNModel()
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mean_squared_error')

# Stage 6: Building and Training the Model
# ... (rest of the code)



# Use the DisjointLoader instead of SingleLoader
from spektral.data.loaders import DisjointLoader
train_loader = DisjointLoader(train_dataset, epochs=50)
new_var = model.fit(train_loader.load(), steps_per_epoch=train_loader.steps_per_epoch)
new_var
# ... (rest of the code)
#

# ... (rest of the code)
# Stage 7: Making Predictions and Evaluating the Model
test_loader = SingleLoader(test_dataset)
predictions = model.predict(test_loader.load(), steps=test_loader.steps_per_epoch)

predictions = scaler.inverse_transform(predictions)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

mse = mean_squared_error(y_test_inv, predictions)
rmse = np.sqrt(mse)
print("RMSE: ", rmse)

first_positive_index = np.argmax(y_test_inv > 0)
print("First positive index in test data: ", first_positive_index)

# Stage 8: Plotting the Results
plt.figure(figsize=(10, 5))
plt.plot(y_test_inv, label='Original SO2 Data')
plt.plot(predictions, label='Predicted SO2 Data')
plt.legend()
plt.title('Original vs Predicted SO2 Data')
plt.xlabel('Sample Index')
plt.ylabel('SO2 Concentration')
plt.show()


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filegqu835ha.py", line 10, in tf__call
        (x, a) = ag__.ld(inputs)

    ValueError: Exception encountered when calling layer '